# Implementation and Analysis of the QR Algorithm for Eigenvalue Computation

This notebook implements the QR algorithm for computing eigenvalues of real symmetric matrices:

- Matrices are reduced to Hessenberg form using **Householder reflections**.
- Eigenvalues are computed via **unshifted QR iteration**.
- Two stopping criteria are tested:
  - **Fixed iteration count**
  - **Epsilon-based convergence** on the off-diagonal norm
- Results are compared to NumPy's built-in eigenvalue solver in terms of:
  - Runtime
  - Accuracy (L₂-norm of eigenvalue difference)
- The notebook also includes a small experiment on **nonsymmetric matrices**.


In [12]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
np.random.seed(0)

# Plot style
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["font.size"] = 12
plt.rcParams["axes.grid"] = True



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\user\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
def householder_hessenberg(A: np.ndarray) -> np.ndarray:
    """
    Reduce a real matrix A to upper Hessenberg form H using Householder reflections:
    H = Q^T A Q.

    For symmetric matrices, this yields a tridiagonal matrix, which accelerates QR iteration.
    """
    A = A.astype(float).copy()
    n = A.shape[0]

    for k in range(n - 2):
        x = A[k + 1 :, k]
        if x.size == 0:
            break

        # If already almost zero below the first entry, skip
        if np.allclose(x[1:], 0.0):
            continue

        e1 = np.zeros_like(x)
        e1[0] = 1.0

        alpha = np.linalg.norm(x)
        if alpha == 0.0:
            continue

        sign = 1.0 if x[0] >= 0.0 else -1.0
        v = x + sign * alpha * e1
        v_norm = np.linalg.norm(v)
        if v_norm == 0.0:
            continue
        v /= v_norm

        # Apply reflection from the left
        A[k + 1 :, k:] -= 2.0 * np.outer(v, v @ A[k + 1 :, k:])

        # Apply reflection from the right
        A[:, k + 1 :] -= 2.0 * np.outer(A[:, k + 1 :] @ v, v)

    return A


In [ ]:
def householder_qr(A: np.ndarray):
    """
    QR factorization of A using Householder reflections.
    Returns Q, R such that A = Q @ R, Q orthogonal, R upper triangular.
    """
    A = A.astype(float).copy()
    n = A.shape[0]
    Q = np.eye(n)
    R = A

    for k in range(n - 1):
        x = R[k:, k]
        if x.size == 0:
            break

        # If all but first are ~0, no need to reflect
        if np.allclose(x[1:], 0.0):
            continue

        e1 = np.zeros_like(x)
        e1[0] = 1.0

        alpha = np.linalg.norm(x)
        if alpha == 0.0:
            continue

        sign = 1.0 if x[0] >= 0.0 else -1.0
        v = x + sign * alpha * e1
        v_norm = np.linalg.norm(v)
        if v_norm == 0.0:
            continue
        v /= v_norm

        H_small = np.eye(len(x)) - 2.0 * np.outer(v, v)
        H = np.eye(n)
        H[k:, k:] = H_small

        R = H @ R
        Q = Q @ H.T  # H is symmetric, but transpose is explicit

    return Q, R


In [13]:
def qr_eigenvalues(
    A: np.ndarray,
    max_iter: int = 1000,
    tol: float | None = None,
    relative: bool = True,
):
    """
    Compute eigenvalues of A using unshifted QR iteration.

    Parameters
    ----------
    A : ndarray (n, n)
        Real input matrix (ideally symmetric or Hessenberg).
    max_iter : int
        Maximum number of QR iterations.
    tol : float or None
        If None, perform exactly max_iter iterations.
        If not None, stop when the Frobenius norm of the strict lower triangle
        is below tol (either absolute or relative).
    relative : bool
        If True, use relative condition: offdiag_norm / ||A||_F < tol.
        If False, use absolute condition: offdiag_norm < tol.

    Returns
    -------
    evals : ndarray (n,)
        Approximated eigenvalues, sorted in descending order.
    iters : int
        Number of QR iterations performed.
    Ak : ndarray (n, n)
        Final iterated matrix (approximate Schur form).
    """
    Ak = A.astype(float).copy()
    iters = 0

    for it in range(max_iter):
        iters = it + 1
        Q, R = householder_qr(Ak)
        Ak = R @ Q

        if tol is not None:
            off = np.linalg.norm(np.tril(Ak, k=-1), ord="fro")
            if relative:
                denom = np.linalg.norm(Ak, ord="fro")
                if denom == 0.0:
                    break
                if off / denom < tol:
                    break
            else:
                if off < tol:
                    break

    evals = np.sort(np.diag(Ak))[::-1]
    return evals, iters, Ak


In [14]:
def generate_symmetric_matrix(n: int, kind: str = "float") -> np.ndarray:
    """
    Generate an n x n random symmetric matrix of a given kind.
    kind ∈ {"float", "int", "int_100_1000"}.
    """
    if kind == "float":
        M = np.random.randn(n, n)
    elif kind == "int":
        M = np.random.randint(-10, 11, size=(n, n))
    elif kind == "int_100_1000":
        M = np.random.randint(100, 1001, size=(n, n))
    else:
        raise ValueError(f"Unknown matrix kind: {kind}")

    A = (M + M.T) / 2.0
    return A.astype(float)


def eigenvalues_builtin(A: np.ndarray) -> np.ndarray:
    """
    Reference eigenvalues for symmetric matrices using NumPy's eigvalsh.
    """
    return np.linalg.eigvalsh(A)


def eigen_error(lambda_custom: np.ndarray, lambda_ref: np.ndarray) -> float:
    """
    L2 norm of difference between two eigenvalue vectors (sorted descending).
    """
    lc = np.sort(lambda_custom)[::-1]
    lr = np.sort(lambda_ref)[::-1]
    return np.linalg.norm(lc - lr)


In [15]:
def run_experiment_one_matrix(
    n: int,
    kind: str,
    mode: str = "fixed",      # "fixed" or "eps"
    fixed_iter: int = 500,
    eps: float = 1e-8,
    max_iter: int = 2000,
    relative_tol: bool = True,
) -> dict:
    """
    Run eigenvalue computation on one random symmetric matrix and compare to NumPy.

    Returns a record with:
    - n, kind, mode
    - runtime_custom, runtime_builtin
    - iters_custom
    - l2_error
    """
    A = generate_symmetric_matrix(n, kind)

    # Built-in solver
    t0 = time.time()
    lam_ref = eigenvalues_builtin(A)
    runtime_builtin = time.time() - t0

    # Our method: first reduce to Hessenberg form
    t1 = time.time()
    H = householder_hessenberg(A)

    if mode == "fixed":
        lam_custom, iters, _ = qr_eigenvalues(
            H, max_iter=fixed_iter, tol=None
        )
    elif mode == "eps":
        lam_custom, iters, _ = qr_eigenvalues(
            H, max_iter=max_iter, tol=eps, relative=relative_tol
        )
    else:
        raise ValueError("mode must be 'fixed' or 'eps'")

    runtime_custom = time.time() - t1
    err = eigen_error(lam_custom, lam_ref)

    return {
        "n": n,
        "kind": kind,
        "mode": mode,
        "runtime_custom": runtime_custom,
        "runtime_builtin": runtime_builtin,
        "iters_custom": iters,
        "l2_error": err,
    }


In [16]:
np.random.seed(1)
A_example = generate_symmetric_matrix(5, kind="float")

lam_ref = eigenvalues_builtin(A_example)

H_example = householder_hessenberg(A_example)
lam_custom, iters_example, _ = qr_eigenvalues(
    H_example, max_iter=500, tol=1e-10, relative=True
)

print("Matrix A:\n", A_example)
print("\nBuilt-in eigenvalues (descending):")
print(np.sort(lam_ref)[::-1])

print("\nQR eigenvalues (descending):")
print(np.sort(lam_custom)[::-1])

print(f"\nIterations used: {iters_example}")
print(f"L2 error: {eigen_error(lam_custom, lam_ref):.3e}")


Matrix A:
 [[ 1.62434536 -1.45664756  0.46696809 -1.08642994 -0.11760577]
 [-1.45664756  1.74481176 -1.41067381  0.07330544  0.44767667]
 [ 0.46696809 -1.41067381 -0.3224172  -0.63095639  1.01768008]
 [-1.08642994  0.07330544 -0.63095639  0.04221375  0.54265478]
 [-0.11760577  0.44767667  1.01768008  0.54265478  0.90085595]]

Built-in eigenvalues (descending):
[ 3.85841896  1.53313158  0.75220935 -0.24245396 -1.91149631]

QR eigenvalues (descending):
[ 3.85841896  1.53313158  0.75220935 -0.24245396 -1.91149631]

Iterations used: 500
L2 error: 2.121e-14


In [ ]:
# Experiment parameters (tweak these if runtimes are too big)
sizes = [60,70,80,90,100]                  # matrix sizes n
matrix_kinds = ["float", "int", "int_100_1000"]

repeats_fixed = 5   # matrices per (kind, n) for fixed-iteration
repeats_eps   = 5   # matrices per (kind, n) for epsilon-based

fixed_iterations = 500          # fixed stopping: exactly 500 iterations
eps_tolerance    = 1e-8         # relative off-diagonal tolerance
eps_max_iter     = 2000         # max iterations for epsilon stopping

records = []

for kind in matrix_kinds:
    for n in sizes:
        # Fixed-iteration runs
        for _ in range(repeats_fixed):
            rec_fixed = run_experiment_one_matrix(
                n=n,
                kind=kind,
                mode="fixed",
                fixed_iter=fixed_iterations,
            )
            records.append(rec_fixed)

        # Epsilon-based runs
        for _ in range(repeats_eps):
            rec_eps = run_experiment_one_matrix(
                n=n,
                kind=kind,
                mode="eps",
                eps=eps_tolerance,
                max_iter=eps_max_iter,
                relative_tol=True,
            )
            records.append(rec_eps)

df = pd.DataFrame(records)
df.head()


In [ ]:
df_fixed = df[df["mode"] == "fixed"].reset_index(drop=True)
df_eps   = df[df["mode"] == "eps"].reset_index(drop=True)

df.to_csv("results_all.csv", index=False)
df_fixed.to_csv("results_fixed.csv", index=False)
df_eps.to_csv("results_eps.csv", index=False)

df_fixed.head(), df_eps.head()


In [ ]:
def plot_runtime_comparison(df_mode: pd.DataFrame, kind: str, mode: str, title_suffix: str):
    sub = df_mode[df_mode["kind"] == kind]
    grp = (
        sub.groupby("n")[["runtime_custom", "runtime_builtin"]]
        .mean()
        .reset_index()
    )

    plt.figure()
    plt.plot(grp["n"], grp["runtime_custom"], marker="o", label="Our QR")
    plt.plot(grp["n"], grp["runtime_builtin"], marker="o", label="Built-in")
    plt.xlabel("Matrix size n")
    plt.ylabel("Runtime (seconds)")
    plt.title(f"Runtime ({mode}) — {title_suffix}")
    plt.legend()
    plt.show()


def plot_accuracy(df_mode: pd.DataFrame, kind: str, mode: str, title_suffix: str):
    sub = df_mode[df_mode["kind"] == kind]
    grp = sub.groupby("n")[["l2_error"]].mean().reset_index()

    plt.figure()
    plt.plot(grp["n"], grp["l2_error"], marker="o")
    plt.xlabel("Matrix size n")
    plt.ylabel(r"$\|\lambda_{\mathrm{QR}} - \lambda_{\mathrm{builtin}}\|_2$")
    plt.title(f"Accuracy ({mode}) — {title_suffix}")
    plt.show()


def kind_label(kind: str) -> str:
    if kind == "float":
        return "Floating-point"
    if kind == "int":
        return "Integer"
    if kind == "int_100_1000":
        return "Integer [100, 1000]"
    return kind


In [ ]:
for kind in matrix_kinds:
    plot_runtime_comparison(
        df_fixed,
        kind=kind,
        mode="Fixed 500 iterations",
        title_suffix=kind_label(kind),
    )


In [ ]:
for kind in matrix_kinds:
    plot_runtime_comparison(
        df_eps,
        kind=kind,
        mode=f"Epsilon stopping (tol={eps_tolerance}, max_iter={eps_max_iter})",
        title_suffix=kind_label(kind),
    )


In [ ]:
for kind in matrix_kinds:
    plot_accuracy(
        df_fixed,
        kind=kind,
        mode="Fixed 500 iterations",
        title_suffix=kind_label(kind),
    )


In [ ]:
for kind in matrix_kinds:
    plot_accuracy(
        df_eps,
        kind=kind,
        mode=f"Epsilon stopping (tol={eps_tolerance}, max_iter={eps_max_iter})",
        title_suffix=kind_label(kind),
    )


In [ ]:
def generate_nonsymmetric_matrix(n: int, kind: str = "float") -> np.ndarray:
    """
    Generate a random nonsymmetric matrix of a given kind.
    """
    if kind == "float":
        A = np.random.randn(n, n)
    elif kind == "int":
        A = np.random.randint(-10, 11, size=(n, n)).astype(float)
    else:
        raise ValueError("Unsupported kind for nonsymmetric test.")
    return A.astype(float)


def run_nonsymmetric_experiment(
    n_list = [60, 120, 180],
    repeats: int = 3,
    fixed_iter: int = 500,
    eps: float = 1e-8,
    eps_max_iter: int = 5000,
) -> pd.DataFrame:
    records = []

    for n in n_list:
        for _ in range(repeats):
            A = generate_nonsymmetric_matrix(n, kind="float")

            # Built-in general eigen solver
            t0 = time.time()
            _ = np.linalg.eigvals(A)
            runtime_builtin = time.time() - t0

            # Our method: fixed iterations
            t1 = time.time()
            H = householder_hessenberg(A)
            lam_fixed, it_fixed, _ = qr_eigenvalues(
                H, max_iter=fixed_iter, tol=None
            )
            runtime_fixed = time.time() - t1

            # Our method: epsilon-based
            t2 = time.time()
            H2 = householder_hessenberg(A)
            lam_eps, it_eps, _ = qr_eigenvalues(
                H2, max_iter=eps_max_iter, tol=eps, relative=True
            )
            runtime_eps = time.time() - t2

            records.append(
                {
                    "n": n,
                    "runtime_builtin": runtime_builtin,
                    "runtime_fixed": runtime_fixed,
                    "runtime_eps": runtime_eps,
                    "iters_fixed": it_fixed,
                    "iters_eps": it_eps,
                }
            )

    return pd.DataFrame(records)


df_nonsym = run_nonsymmetric_experiment()
df_nonsym.head()


In [ ]:
df_nonsym.to_csv("results_nonsym.csv", index=False)

grp_nonsym = df_nonsym.groupby("n").mean().reset_index()

plt.figure()
plt.plot(grp_nonsym["n"], grp_nonsym["runtime_builtin"], marker="o", label="Built-in eig")
plt.plot(grp_nonsym["n"], grp_nonsym["runtime_fixed"], marker="o", label="Our QR (fixed)")
plt.plot(grp_nonsym["n"], grp_nonsym["runtime_eps"], marker="o", label="Our QR (epsilon)")
plt.xlabel("Matrix size n")
plt.ylabel("Runtime (seconds)")
plt.title("Runtime Comparison — Nonsymmetric Matrices")
plt.legend()
plt.show()

grp_nonsym


## Summary

This notebook implemented a self-contained QR algorithm for eigenvalue computation:

- Matrices were reduced to Hessenberg form using **Householder reflections**.
- Eigenvalues were computed via **unshifted QR iteration** under:
  - A **fixed iteration** stopping rule.
  - An **epsilon-based convergence** rule using the relative Frobenius norm of the strict lower triangle.
- The implementation was validated on random symmetric matrices of different types:
  - Floating-point entries
  - Small integer entries
  - Larger integers in [100, 1000]
- For each case, we compared:
  - **Runtime** of our QR method vs NumPy's built-in solver.
  - **Accuracy** via the L₂-norm difference between eigenvalues.

Additional experiments on **nonsymmetric matrices** demonstrated that the basic unshifted QR algorithm is inefficient in that setting, especially with strict convergence criteria, whereas NumPy's built-in solver remains very fast.
